# 퍼셉트론

In [1]:
import tensorflow as tf
import numpy as np

이번엔 Iris 데이터 중 두 종류를 분류하는 퍼셉트론을 제작한다. y값은 1 또는 -1을 사용하고 활성화 함수로는 하이퍼탄젠트(hypertangent)함수를 사용한다.(시그모이드를 사용하면 미분값이 0으로 되는 경우가 생김)

$$\Large{ \hat{y} = tanh(w^Tx) }$$

비용 함수로는 다음 식을 사용한다. (맞는것은 빼고 틀린 것만 더하겠다)

$$
\large{
Loss = \sum_{i=1}^N \max(0, -y_i \hat{y_i})
}
$$

y와 y_hat이 동일하면 곱하기 값이 -1이 나와서 0을 반환
```
  1, 1 =>  1 * 1 * -1 => -1
 -1,-1 => -1 * -1 * -1 => -1
```

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [3]:
idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis] 

In [4]:
X_data.shape, y_data.shape

((100, 2), (100, 1))

#### 이 데이터를 이용해 Perceptron을 구현해보세요! 
 - 데이터 하나당 한번씩 weights 업데이트
 - step size == 0.0003
 - iteration == 200

In [5]:
num_iter = 500
lr = 0.0003

-----

In [6]:
w = tf.Variable(tf.random.normal([2, 1], dtype=tf.float64))
b = tf.Variable(tf.random.normal([1, 1], dtype=tf.float64))

In [7]:
zero = tf.constant(0, dtype=tf.float64)

for i in range(num_iter):
    for i in range(X_data.shape[0]): # X 데이터 개수 만큼
        x = X_data[i:i+1] # == expand_dims 랑 같은 효과
        y = y_data[i:i+1]
        
        with tf.GradientTape() as tape:
            logit = tf.matmul(x, w) + b   # logit = 우리의 목표 함수
            y_hat = tf.tanh(logit)        # 활성화 함수를 거쳐 예측 (하이퍼탄젠트)
            loss = tf.maximum(zero, tf.multiply(-y, y_hat)) # 데이터가 한개씩이라 reduce_max안해도됨 

        grad = tape.gradient(loss, [w, b])
        w.assign_add(-lr * grad[0])
        b.assign_add(-lr * grad[1])


In [10]:
# 위에서 예측한 W와 b로 함수를 구현
y_pred = tf.tanh(tf.matmul(X_data, w) + b)

In [11]:
print("예측치 :", y_pred[0].numpy(),  "정답 :", y_data[0])
print("예측치 :", y_pred[51].numpy(), "정답 :", y_data[51])
print("예측치 :", y_pred[88].numpy(), "정답 :", y_data[88])

예측치 : [0.99999998] 정답 : [-1.]
예측치 : [1.] 정답 : [1.]
예측치 : [1.] 정답 : [1.]


#### 0보다크면 1, 0보다 작으면 -1 로 후처리 작업

In [12]:
print("예측치 :", -1 if y_pred[0] < 0 else 1,  "정답 :", y_data[0])
print("예측치 :", -1 if y_pred[51] < 0 else 1, "정답 :", y_data[51])
print("예측치 :", -1 if y_pred[88] < 0 else 1, "정답 :", y_data[88])

예측치 : 1 정답 : [-1.]
예측치 : 1 정답 : [1.]
예측치 : 1 정답 : [1.]
